In [159]:
import quandl
import pandas as pd
import numpy as np
import sys
from math import sqrt
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook()
from scipy.interpolate import interp1d
from bokeh.models import HoverTool

Loading BokehJS ...

In [160]:
# The list of stocks included in the query
#
stock_list = ['AAPL', 'MCD', 'BK','STZ','JNJ','DIS','MSFT','FB','KO','UNP','NFLX','GS','AMZN','BRK.B']
num_trade_dys_py = 250
num_random_portfolios = 50000

In [161]:
# The quandl api call will return the historical pricing information and stores the data in a table
# The columns I've requested are the ticker, the closing price and the associated date
#
quandl.ApiConfig.api_key = 'ENTER KEY HERE'
raw_data=quandl.get_table("SHARADAR/SEP", qopts={"columns":['ticker','date','close']},ticker=stock_list, paginate=True)

# The data is pivoted so that the column headings are the ticker symbols and the date is the index
#
quandl_data = raw_data.pivot(index='date',columns='ticker', values='close')

In [162]:
# Some quotes are not found in quandl. I needed to download pricing data from yahoo and upload as a csv
#
external_quotes = pd.read_csv('C:/Users/ryanj/python projects/missing_stock_quotes.csv')
external_quotes['date'] = external_quotes['date'].astype('datetime64[ns]')

# Remove the spaces around the column names
#
external_quotes.columns = external_quotes.columns.str.replace(' ', '')

# Change the index to date
#
external_quotes.set_index('date', inplace=True)

# The quote 'VOO' is the S&P index I will use VOO as my proxy for market risk / return
# I will split the dataframe into two pieces; the missing_quotes and the market_portfolio
market_portfolio = external_quotes[['VOO']]
missing_quotes = external_quotes.loc[:, ~external_quotes.columns.isin(['VOO'])]

In [163]:
# Merge the quandl dataframe with the csv file
#
data = pd.merge(quandl_data, missing_quotes, on='date')

# The number of ticker symbols in DataFrame data
#
num_assets = data.shape[1]

In [164]:
# calculate the daily and annual return of the stocks included in the stock list
#
daily_sec_return = data.pct_change()

# Annualize the daily returns
#
annual_sec_return = daily_sec_return.mean() * num_trade_dys_py
annual_sec_return.shape

(23,)

In [165]:
# Calculate the covariance between stocks in the portfolio based on the daily return and then annualize the covariance
# by multiplying by the number of estimated trading days
# covariance will measure the degree that stocks included in the portfolio move in tandem
# negative covaried stocks move in opposite directions while positively covaried stocks move in the same direction
daily_sec_covariance = daily_sec_return.cov()
annual_sec_covariance = daily_sec_covariance * num_trade_dys_py

# Annualize the daily std deviation
#
annual_sec_std_deviation = (daily_sec_return.std()) * np.sqrt(num_trade_dys_py)
annual_sec_covariance.head()

,AAPL,AMZN,BK,BRK.B,DIS,FB,GS,JNJ,KO,MCD,...,UNP,BAM,ATD.B,GLD,VSS,CQQQ,VNQI,VNQ,XAR,JRO
AAPL,0.059090,0.032223,0.017159,0.017543,0.016954,0.029931,0.023956,0.011490,0.007520,0.010490,...,0.020767,0.017294,0.002283,-0.003654,0.015834,0.028374,0.013491,0.009624,0.018902,0.005081
AMZN,0.032223,0.088254,0.019325,0.017791,0.018809,0.048070,0.024683,0.013032,0.008590,0.012565,...,0.020076,0.019761,-0.000409,-0.003964,0.018658,0.035724,0.016686,0.011819,0.019805,0.005698
BK,0.017159,0.019325,0.045407,0.022406,0.016552,0.018726,0.032726,0.011072,0.007574,0.010145,...,0.022938,0.016268,0.003140,-0.007101,0.015146,0.022946,0.013231,0.007152,0.019235,0.004887
BRK.B,0.017543,0.017791,0.022406,0.025867,0.015649,0.016540,0.024955,0.012937,0.009677,0.010384,...,0.020581,0.014779,0.001813,-0.004069,0.012746,0.017876,0.011426,0.009297,0.016545,0.003172
DIS,0.016954,0.018809,0.016552,0.015649,0.038165,0.017813,0.019668,0.010189,0.007828,0.009411,...,0.017056,0.014181,0.001161,-0.003137,0.012291,0.018444,0.011255,0.009815,0.014773,0.003980


In [166]:
# Generate an array where the number of rows equal num_random_portfolios and where the total number of columns will tie to the number of stocks in the portfolio
# the columns will total 1
#
X = np.random.dirichlet(np.ones(num_assets),size=num_random_portfolios)

In [167]:
# Convert the array to a dataframe and assign the ticker symbol to the column name
#
rand_port_wts = pd.DataFrame(X, columns=data.columns)
rand_port_wts.shape

(50000, 23)

In [168]:
# multiply the random portfolio weights by the annualized return
#
portfolio_return = rand_port_wts.dot(annual_sec_return)
portfolio_return = pd.DataFrame(portfolio_return, columns=['portfolio_return'])

In [169]:
# Function to pass one row the rate frame to calculation
rand_port_wts.head()

,AAPL,AMZN,BK,BRK.B,DIS,FB,GS,JNJ,KO,MCD,...,UNP,BAM,ATD.B,GLD,VSS,CQQQ,VNQI,VNQ,XAR,JRO
0,0.098492,0.118874,0.039748,0.031300,0.010830,0.026059,0.037010,0.016364,0.017940,0.002619,...,0.008499,0.055173,0.147436,0.108514,0.010182,0.024586,0.010254,0.034255,0.084099,0.015680
1,0.012835,0.012963,0.023185,0.048750,0.014135,0.077657,0.007626,0.038778,0.064021,0.040375,...,0.011231,0.047427,0.014422,0.022922,0.028135,0.088881,0.204128,0.049895,0.069163,0.059061
2,0.113518,0.022362,0.060524,0.018217,0.080046,0.056070,0.092652,0.171039,0.017590,0.008708,...,0.005894,0.009857,0.012617,0.014858,0.013250,0.038266,0.039875,0.062874,0.102229,0.013369
3,0.015761,0.082968,0.005584,0.035940,0.002169,0.066557,0.127134,0.018055,0.023350,0.077482,...,0.002121,0.024680,0.001644,0.070044,0.007136,0.061423,0.046855,0.012741,0.065976,0.016253
4,0.045707,0.124425,0.029518,0.064502,0.051685,0.015747,0.069823,0.081787,0.014113,0.010309,...,0.044839,0.067333,0.041056,0.037374,0.000761,0.019542,0.036659,0.116372,0.000955,0.008355


In [177]:
portfolio_std_deviation = pd.DataFrame(rand_port_wts.apply(lambda x: np.sqrt(np.dot(x.T, np.dot(annual_sec_covariance, x))), axis=1),columns=['portfolio_std_dev'])

In [182]:
portfolio_std_deviation.columns[0]

'port_std_dev'

In [203]:
portfolio_data = pd.DataFrame(rand_port_wts)

In [204]:
portfolio_data['return'] = portfolio_return.portfolio_return
portfolio_data['std_dev'] = portfolio_std_deviation.port_std_dev
portfolio_data['sharpe_ratio'] = portfolio_data['return'] / portfolio_data['std_dev']
portfolio_data

,AAPL,AMZN,BK,BRK.B,DIS,FB,GS,JNJ,KO,MCD,...,GLD,VSS,CQQQ,VNQI,VNQ,XAR,JRO,return,std_dev,sharpe_ratio
0,0.098492,0.118874,0.039748,0.031300,0.010830,0.026059,0.037010,0.016364,0.017940,0.002619,...,0.108514,0.010182,0.024586,0.010254,0.034255,0.084099,0.015680,0.168418,0.115697,1.455679
1,0.012835,0.012963,0.023185,0.048750,0.014135,0.077657,0.007626,0.038778,0.064021,0.040375,...,0.022922,0.028135,0.088881,0.204128,0.049895,0.069163,0.059061,0.112179,0.117918,0.951333
2,0.113518,0.022362,0.060524,0.018217,0.080046,0.056070,0.092652,0.171039,0.017590,0.008708,...,0.014858,0.013250,0.038266,0.039875,0.062874,0.102229,0.013369,0.130768,0.127260,1.027568
3,0.015761,0.082968,0.005584,0.035940,0.002169,0.066557,0.127134,0.018055,0.023350,0.077482,...,0.070044,0.007136,0.061423,0.046855,0.012741,0.065976,0.016253,0.161781,0.138464,1.168401
4,0.045707,0.124425,0.029518,0.064502,0.051685,0.015747,0.069823,0.081787,0.014113,0.010309,...,0.037374,0.000761,0.019542,0.036659,0.116372,0.000955,0.008355,0.166981,0.138555,1.205166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.129084,0.003487,0.033930,0.016970,0.023480,0.000702,0.035467,0.010200,0.028180,0.101462,...,0.024934,0.029778,0.028455,0.137592,0.027456,0.114576,0.014621,0.138486,0.126203,1.097333
49996,0.003719,0.069581,0.025405,0.079291,0.019879,0.014036,0.040388,0.087998,0.127260,0.000616,...,0.041884,0.020941,0.004706,0.041356,0.002562,0.003267,0.001501,0.168226,0.137283,1.225395
49997,0.124341,0.039390,0.007114,0.028443,0.149634,0.061605,0.011007,0.005099,0.051803,0.041094,...,0.024089,0.024762,0.007078,0.007170,0.024180,0.020308,0.006645,0.187664,0.132936,1.411681
49998,0.006669,0.223689,0.008854,0.016096,0.180984,0.005533,0.004646,0.001652,0.003843,0.082247,...,0.016773,0.012332,0.003917,0.003702,0.002712,0.057914,0.065110,0.186604,0.144625,1.290262


In [211]:
portfolio_data.loc[portfolio_data.sharpe_ratio.idxmax()]

AAPL            0.013076
AMZN            0.059941
BK              0.018734
BRK.B           0.019202
DIS             0.022255
FB              0.040326
GS              0.005695
JNJ             0.001359
KO              0.003094
MCD             0.214186
MSFT            0.085867
NFLX            0.006648
STZ             0.039881
UNP             0.074707
BAM             0.001346
ATD.B           0.234717
GLD             0.059876
VSS             0.011427
CQQQ            0.009001
VNQI            0.006331
VNQ             0.031916
XAR             0.005780
JRO             0.034636
return          0.171017
std_dev         0.108039
sharpe_ratio    1.582910
Name: 36388, dtype: float64